In [11]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np
import pandas as pd

In [12]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [13]:
len(tf.config.list_physical_devices('GPU'))

1

In [14]:
def process_csv_file(file):
    try:
     csv_data = pd.read_csv(file)
    except:
         print("CANT READ CSV: ", file)
         return
    
    if csv_data.isnull().values.any():
            
            return False
    try:
        csv_data = csv_data.drop("class", axis = 1)
    except KeyError:
        pass
    return (csv_data.to_numpy())

In [15]:
!cd /Tmp/ASL-data/Database-augmented/coffee/; ls;
!ls

/bin/bash: line 1: cd: /Tmp/ASL-data/Database-augmented/coffee/: No such file or directory
 capture.txt   example.txt	 lstm_25_labels.ipynb   lstm-testing-model.py
 demo_test    'lstm (1).ipynb'	 lstm.ipynb
 capture.txt   example.txt	 lstm_25_labels.ipynb   lstm-testing-model.py
 demo_test    'lstm (1).ipynb'	 lstm.ipynb


In [16]:
!cd /Tmp/linxinle/ASL-data/Database-augmented/

/bin/bash: line 1: cd: /Tmp/linxinle/ASL-data/Database-augmented/: No such file or directory


In [17]:
# file = "/Tmp/linxinle/ASL-data/Database-augmented/test/coffee/2sGQuduhAf41354.csv"
# file = "/media/makishima/DATA1/Personnel/Other learning/Programming/Personal_projects/ASL_Language_translation/training_models/mediapipe/Database-augmented/test/coffee/2sGQuduhAf41354.csv"
file = "/Tmp/shariffa/ASL-words-translator/6_Databases/Aug25LabelsData/train/book/41X2t_s2Ai40.csv"
process_csv_file(file)

array([[ 0.48266143,  0.25151151, -0.66583532, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.48195514,  0.25146753, -0.66424268, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.48045793,  0.25147596, -0.65664881, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.48019916,  0.24556774, -0.64471287, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.48011598,  0.24555989, -0.65619022, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.48010522,  0.24555062, -0.66737396, ...,  0.        ,
         0.        ,  0.        ]])

In [18]:
def set_padding(arr, max_length):
    arr = np.append(arr, np.zeros((max_length-arr.shape[0],300)), axis=0)
    return np.expand_dims(arr, 0)

In [19]:
def prepare_data(folder_path, labels, train=True):
    total = 0
    max_length = 211
    
    # find max length
    # for folder in os.listdir(folder_path):
    #     print(folder)
    #     if folder != ".DS_Store":
    #         total += len(os.listdir(os.path.join(folder_path, folder)))
    #         for file in os.listdir(os.path.join(folder_path, folder)):
    #             if file != ".DS_Store":
    #                 data = process_csv_file(os.path.join(folder_path, folder, file))
    #                 if data is not(False):
    #                     if data.shape[0] > max_length:
    #                         max_length = data.shape[0]
    # print(max_length)
    # Make the arrays
    X = np.empty((0, max_length, 300))
    Y = np.empty((0,), int)
    print("----------")
    for folder in os.listdir(folder_path):
        print(folder)
        if folder != ".DS_Store":
            i=0
            for file in os.listdir(os.path.join(folder_path, folder)):
                i+=1
                if file != ".DS_Store": 
                    data = process_csv_file(os.path.join(folder_path, folder, file))
                    if data is not(False):
                        data = set_padding(data, max_length)
                        X = np.append(X, data, axis=0)
                        Y = np.append(Y, labels[folder])
                        if (i% 50 == 0):
                            print(i)
                if i == 5:
                    break
                    
    print(total, max_length)
    return X, Y

In [20]:
# get labels
def get_labels(folder_path):
    labels = {}
    count = 0
    for folder in os.listdir(folder_path):
            labels[folder] = count
            count += 1
    return labels

labels = get_labels("/Tmp/shariffa/ASL-words-translator/6_Databases/Aug25LabelsData/train")
labels

{'hear': 0,
 'music': 1,
 'bookstore': 2,
 'classroom': 3,
 'doctor': 4,
 'focus': 5,
 'chicken': 6,
 'door': 7,
 'many': 8,
 'polite': 9,
 'good morning': 10,
 'coffee': 11,
 'photographer': 12,
 'hamburger': 13,
 'i': 14,
 'phone': 15,
 'brother': 16,
 'i love you': 17,
 'milk': 18,
 'dog': 19,
 'ocean': 20,
 'research': 21,
 'book': 22,
 'open': 23,
 'money': 24}

In [21]:
folder_path_train = "/Tmp/shariffa/ASL-words-translator/6_Databases/Aug25LabelsData/train"
# folder_path_train="/media/makishima/DATA1/Personnel/Other learning/Programming/Personal_projects/ASL_Language_translation/training_models/mediapipe/data_25_labels_augmentation/train/"
X_train, Y_train = prepare_data(folder_path_train, labels)

----------
hear
music
bookstore
classroom
doctor
focus
chicken
door
many
polite
good morning
coffee
photographer
hamburger
i
phone
brother
i love you
milk
dog
ocean
research
book
open
money
0 211


In [22]:
# folder_path_test = "/Tmp/ASL-data/Database-augmented/test"
folder_path_test = "/Tmp/linxinle/ASL-data/Database-augmented/test"
folder_path_test = "/Tmp/shariffa/ASL-words-translator/6_Databases/Aug25LabelsData/train"
# labels = {"coffee": 0, 'dog': 1, 'milk': 2, 'door': 3}
X_test, Y_test = prepare_data(folder_path_test, labels, train=False)

----------
hear
music
bookstore
classroom
doctor
focus
chicken
door
many
polite
good morning
coffee
photographer
hamburger
i
phone
brother
i love you
milk
dog
ocean
research
book
open
money
0 211


In [25]:
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    [
    [711, 632, 71],
    [73, 8, 3215, 55, 927],
    [83, 91, 1, 645, 1253, 927],
], padding="post"
)
# print(X[0][1])
print(((padded_inputs)))

[[ 711  632   71    0    0    0]
 [  73    8 3215   55  927    0]
 [  83   91    1  645 1253  927]]


In [26]:
# Y = np.expand_dims(Y, axis=0)
# Y= Y.T


In [23]:
# shuffle data
from sklearn.utils import shuffle

X_train,Y_train = shuffle(X_train, Y_train, random_state=0)
X_val,Y_val = shuffle(X_test, Y_test, random_state=0)
X_train, Y_train

(array([[[ 0.57075262,  0.23228616, -0.7571311 , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.57188809,  0.2330122 , -0.78263861, ...,  0.        ,
           0.        ,  0.        ],
         [ 0.57207823,  0.23495021, -0.80215025, ...,  0.        ,
           0.        ,  0.        ],
         ...,
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ]],
 
        [[ 0.52638198,  0.26888801, -0.82134467, ...,  0.        ,
           0.        ,  0.        ],
         [ 0.52746353,  0.2689094 , -0.84388655, ...,  0.        ,
           0.        ,  0.        ],
         [ 0.52761596,  0.26939814, -0.86171532, ...,  0.        ,
           0.        ,  0.        ],
         ...,
         [ 0.        ,  0.       

In [24]:
X_train.shape, Y_train.shape

((118, 211, 300), (118,))

In [25]:
x_shape = X_train.shape

In [26]:
#LSTM model - 4 labels
from tensorflow.keras import layers
x_shape = (457, 211, 300)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten

model_lstm = Sequential()
model_lstm.add(LSTM(64, return_sequences=True, input_shape=(211, 300)))
# model_lstm.add(Dropout(0.2))
# model_lstm.add(LSTM(128, return_sequences=True))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(64))
model_lstm.add(Flatten())
model_lstm.add(Dense(64, activation="relu"))
model_lstm.add(Dense(64, activation="relu"))
model_lstm.add(Dense(4, activation='softmax'))


2023-07-07 23:21:38.239567: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-07 23:21:38.239830: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-07 23:21:38.240005: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [27]:
#LSTM model - 25 labels
from tensorflow.keras import layers
x_shape = (457, 211, 300)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten

model_lstm = Sequential()
model_lstm.add(LSTM(64, return_sequences=True, input_shape=(211, 300)))
# model_lstm.add(Dropout(0.2))
# model_lstm.add(LSTM(128, return_sequences=True))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(64, return_sequences=True))
model_lstm.add(Flatten())
model_lstm.add(Dense(256, activation="relu", kernel_regularizer=keras.regularizers.l1_l2(0.05)))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(128, activation="relu"))
model_lstm.add(Dense(64, activation="relu", kernel_regularizer=keras.regularizers.l1_l2(0.05)))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(64, activation="relu"))
model_lstm.add(Dense(25, activation='softmax'))


2023-07-07 23:21:43.870011: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-07 23:21:43.870733: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-07 23:21:43.871487: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [28]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "/Tmp/shariffa/LSTM-25-labels.{epoch:02d}-{val_accuracy:.2f}",
#     "/Tmp/linxinle/models/LSTM-25-labels.{epoch:02d}-{val_accuracy:.2f}",

    monitor='val_accuracy',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch',
)

In [29]:
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model_lstm.fit(X_train, Y_train, epochs=50, validation_data=(X_val, Y_val), callbacks=[checkpoint])

Epoch 1/50


2023-07-07 23:21:47.203176: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-07 23:21:47.204143: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-07 23:21:47.204810: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

UnknownError: Graph execution error:

Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential_1/lstm_2/PartitionedCall]] [Op:__inference_train_function_7841]